In [1]:
pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

In [3]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Nlp/UAS/train.csv', encoding='latin-1')
df.head()

# df = pd.read_csv('train.csv', sep=', ', delimiter=None, 
#                 header='infer', names=None, 
#                 index_col=None, usecols=None, squeeze=False, 
#                 engine=None)
# df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [5]:
df = df.rename(columns={"highlights":"target_text", "article":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...
...,...,...
287108,summarize: By . James Rush . Former first daug...,Chelsea Clinton said question of running for o...
287109,summarize: An apologetic Vanilla Ice has given...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,summarize: America's most lethal sniper claime...,America's most lethal sniper made comment in i...
287111,summarize: By . Sara Malm . PUBLISHED: . 12:19...,A swarm of more than one million has crossed b...


In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((229690, 2), (57423, 2))

In [7]:
from simplet5 import SimpleT5
model = SimpleT5()

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [8]:
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
model.train(train_df=train_df[:20000], 
            eval_df=test_df[:300],  
            source_max_token_len=512, 
            target_max_token_len=512, 
            batch_size=16, max_epochs=2, use_gpu=True, 
            early_stopping_patience_epochs=2)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

In [9]:
import torch

In [10]:
filepath = "model.pt"

# save the model
torch.save(model, filepath)

In [11]:
model1 = torch.load('model.pt', map_location=torch.device('cpu'))

In [12]:
model1.device= torch.device("cpu")

In [13]:
text_input = """A 38-year-old woman in the US, who was apprehended twice for allegedly trying to jump the White House fence last week, has been arrested for scaling a fence at the Treasury Building. Marci Anderson Wahl of Everett, Washington, was arrested after an alarm sounded at about 2:15 am yesterday when she scaled a fence at the Treasury Building, next to the White House. Police said Wahl has told them she was there to speak to US President Donald Trump, the CNN reported. She was charged with unlawful entry and contempt of court. Wahl was first arrested on March 21 last week for trying to jump the White House fence. Once in custody, it was determined that Wahl had been issued a "stay away" order for the White House complex after the incident. On March 21, officers saw Wahl walking and staring at the White House complex before discovering she had jumped a fence on the south side but got stuck. Officers found her hanging from the inside of the fence by her shoelaces, which were "caught on top of the fence," according to a police report. She was re-arrested on March 24 after officers saw her near Lafayette Park. She was released on her own recognisance after a Saturday court appearance. In the incident, Wahl was charged with contempt of court in violation of a stay away order. She pleaded not guilty last Saturday and was again released on her own recognisance."""

In [14]:
text_to_summarize = """summarize: """ + text_input
predicted_output = model1.predict(text_to_summarize)

In [15]:
predicted_output

['woman in the u.s. charged with unlawful entry and contempt of court. police say she scaled fence at the Treasury Building next to the white house. she was first arrested on march 21 last week for trying to jump the fence.']

In [16]:
model_summaries = []

In [17]:
summary_model = df['target_text'][:20]

In [18]:
summary_model

0     Bishop John Folda, of North Dakota, is taking ...
1     Criminal complaint: Cop used his role to help ...
2     Craig Eccleston-Todd, 27, had drunk at least t...
3     Nina dos Santos says Europe must be ready to a...
4     Fleetwood top of League One after 2-0 win at S...
5     Prime Minister and his family are enjoying an ...
6     NBA star calls for black and Hispanic communit...
7     London Midland service had been pulling into T...
8     Tony Pulis believes Saido Berahino should look...
9     Black box data from Flight 370 could be analyz...
10    The distortion is the result of the Mercator m...
11    Alan Dershowitz has filed defamation suits aga...
12    Sarah Ernhart, the owner of Sarah Beth Photogr...
13    Manchester United beat Southampton 2-1 at St M...
14    U.S. corporations merge with foreign companies...
15    Half of Brits admit to checking work e-mails w...
16    Dencia's comment is hypocritical considering s...
17    Ex-military chiefs suggested .\nstepping u

In [19]:
text_summaries = []
for x in range(20):
    summary = str(summary_model[x])
    summary = summary[:-2]
    print(summary)
    print()
    text_summaries.append(summary)

Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed

Criminal complaint: Cop used his role to help cocaine traffickers .
Ralph Mata, an internal affairs lieutenant, allegedly helped group get guns .
He also arranged to pay two assassins in a murder plot, a complaint alleges

Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .
Was using phone when he veered across road in Yarmouth, Isle of Wight .
Crashed head-on into 28-year-old Rachel Titley's car, who died in hospital .
Police say he would have been over legal drink-drive limit at time of crash .
He was found guilty at Portsmouth Crown Court of causing death by dangerous driving

Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .
Targeting Russia's business community would be one way of sapping their support fo

In [23]:
text_summaries

['Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed',
 'Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murder plot, a complaint alleges',
 "Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital .\nPolice say he would have been over legal drink-drive limit at time of crash .\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving",
 "Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapp

In [20]:
for x in range(20):
    model_summary = model1.predict(df['source_text'][x])
    
    model_summaries.append(model_summary)
    print(model_summaries[x])

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


["the bishop of the fargo Catholic diocese in north. Dakota has exposed potentially hundreds of church members to the hepatitis. A virus in late September and early October. the risk is low, but officials feel it's important to alert people to the possible exposure."]
['mata was an internal affairs lieutenant for the Miami-Dade police department. authorities allege he helped plan a murder plot and get guns. in one instance, he arranged to pay two assassins to kill rival drug dealers.']
['Craig Eccleston-Todd, 27, was driving home from a night at a pub. as he was reading or replying to text he veered across road and hit car. miss titley, 28, died later from her injuries in hospital. police said he had drunk at least three or four pints of beer before getting behind wheel.']
["european leaders must be ready to accept sanctions are a two-way street. targeting Russia's peripatetic business community would be one way of sapping support for Putin. but where Europe should hold fire though is 

In [21]:
model_summaries = model_summaries[:20]

In [22]:
model_summaries

[["the bishop of the fargo Catholic diocese in north. Dakota has exposed potentially hundreds of church members to the hepatitis. A virus in late September and early October. the risk is low, but officials feel it's important to alert people to the possible exposure."],
 ['mata was an internal affairs lieutenant for the Miami-Dade police department. authorities allege he helped plan a murder plot and get guns. in one instance, he arranged to pay two assassins to kill rival drug dealers.'],
 ['Craig Eccleston-Todd, 27, was driving home from a night at a pub. as he was reading or replying to text he veered across road and hit car. miss titley, 28, died later from her injuries in hospital. police said he had drunk at least three or four pints of beer before getting behind wheel.'],
 ["european leaders must be ready to accept sanctions are a two-way street. targeting Russia's peripatetic business community would be one way of sapping support for Putin. but where Europe should hold fire tho

In [26]:
for i in range(20):
  model_summaries[i] = model_summaries[i][0]

In [27]:
model_summaries

["the bishop of the fargo Catholic diocese in north. Dakota has exposed potentially hundreds of church members to the hepatitis. A virus in late September and early October. the risk is low, but officials feel it's important to alert people to the possible exposure.",
 'mata was an internal affairs lieutenant for the Miami-Dade police department. authorities allege he helped plan a murder plot and get guns. in one instance, he arranged to pay two assassins to kill rival drug dealers.',
 'Craig Eccleston-Todd, 27, was driving home from a night at a pub. as he was reading or replying to text he veered across road and hit car. miss titley, 28, died later from her injuries in hospital. police said he had drunk at least three or four pints of beer before getting behind wheel.',
 "european leaders must be ready to accept sanctions are a two-way street. targeting Russia's peripatetic business community would be one way of sapping support for Putin. but where Europe should hold fire though is 

In [28]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from rouge import Rouge

rouge = Rouge()
rouge.get_scores(summary_model, model_summaries, avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.3792830129228589,
  'p': 0.28979853253632315,
  'f': 0.31631756633269165},
 'rouge-2': {'r': 0.17326173740361567,
  'p': 0.11708019389506759,
  'f': 0.1332364833289402},
 'rouge-l': {'r': 0.3509846042169852,
  'p': 0.26957271405352434,
  'f': 0.2940452009016082}}

In [32]:
text_inputt = """The battle has also highlighted an extraordinary rift among Russian forces, with Prigozhin at one point accusing a Russian brigade of abandoning its position in the city and railing several times at the Defense Ministry over a lack of ammunition.

Prigozhin, a former catering boss who has grown in prominence throughout the war, compared the battlefield to a “meat grinder.”

Bakhmut’s fall would be an undoubted boost to Prigozhin, who recently announced his men would pull out entirely because dwindling ammunition supplies and mounting losses meant there was “nothing left to grind the meat with.”"""

In [33]:
predictt = model1.predict(text_inputt)

In [34]:
predictt

['Prigozhin has repeatedly accused the Defense Ministry of abandoning its position in Bakhmut over a lack of ammunition. Prigozhin recently announced his men would pull out entirely because there was “nothing left to grind the meat with.”']